### Homework

#### Manna Toth

* Calculate coefficients (for SARS-CoV-2 treatment) for Calu-3 and A549 cells, and plot them (scatter plot) against each other. This will show us how similar are the response of these cells to infection. You will have to filter for these data (have 2 DataFrames, containing Mock and SARS-CoV-2 infected samples, and either Calu-3 or A549 cell lines), and run a statistical model with only 'Treatment' factor (basically it is a t-test).
* Please upload this notbook (your_name.ipynb) to the Week6 folder (you should have write access to this, if not please let me know)
* install [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) library in R.

#### 1. Loading packages and creating the data frames

In [18]:
#packages
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
import statsmodels.formula.api as smf
from scipy.stats import ttest_ind

import plotly.express as px

In [19]:
#opening the data_stat created during the class (cleaned, log, normalized)
#data_stat_good2 = data_stat_good.copy()
#data_stat_good2.to_csv("data_stat_good2.csv", index=False)
data = pd.read_csv('data_stat_good2.csv')

In [20]:
data.head()

,DDX11L1,WASH7P,LOC729737,LOC100133331,LOC100288069,LINC00115,FAM41C,SAMD11,NOC2L,KLHL17,...,TMSB4Y,NLGN4Y,TTTY14,CD24,BCORP1,KDM5D,EIF1AY,RPS4Y2,Cell,Treatment
0,0.0,0.000004,0.000014,0.000002,0.000002,0.000001,0.000000e+00,1.409524e-06,0.000227,0.000014,...,0.0,0.0,0.0,0.002382,0.0,0.0,0.0,0.0,NHBE,Mock
1,0.0,0.000003,0.000017,0.000003,0.000002,0.000002,0.000000e+00,2.813116e-06,0.000247,0.000019,...,0.0,0.0,0.0,0.002818,0.0,0.0,0.0,0.0,NHBE,Mock
2,0.0,0.000002,0.000010,0.000003,0.000002,0.000002,8.689961e-08,6.951967e-07,0.000222,0.000011,...,0.0,0.0,0.0,0.003150,0.0,0.0,0.0,0.0,NHBE,Mock
3,0.0,0.000005,0.000018,0.000002,0.000003,0.000002,0.000000e+00,3.679207e-06,0.000271,0.000024,...,0.0,0.0,0.0,0.002211,0.0,0.0,0.0,0.0,NHBE,SARS-CoV-2
4,0.0,0.000003,0.000012,0.000003,0.000001,0.000001,1.456679e-07,1.456678e-06,0.000234,0.000011,...,0.0,0.0,0.0,0.002345,0.0,0.0,0.0,0.0,NHBE,SARS-CoV-2


In [21]:
print(data['Cell'].value_counts())
print (data['Treatment'].value_counts())

A549         19
NHBE         10
A549-ACE2     6
Calu3         6
Name: Cell, dtype: int64
Mock          26
SARS-CoV-2    15
Name: Treatment, dtype: int64


In [22]:
#creating A549 and Calu3 filters
filter_A549 = np.in1d(data['Cell'], ['A549'])
filter_Calu = np.in1d(data['Cell'], ['Calu3'])

In [23]:
#filtering A549 and Calu3 cells
data_A549 = data[filter_A549]
data_Calu = data[filter_Calu]

In [24]:
print(data_A549.shape)
print(data_Calu.shape)

(19, 18547)
(6, 18547)


In [25]:
data_Calu

,DDX11L1,WASH7P,LOC729737,LOC100133331,LOC100288069,LINC00115,FAM41C,SAMD11,NOC2L,KLHL17,...,TMSB4Y,NLGN4Y,TTTY14,CD24,BCORP1,KDM5D,EIF1AY,RPS4Y2,Cell,Treatment
28,0.000000e+00,0.000005,0.000013,8.196425e-07,0.000000e+00,8.196425e-07,0.000000e+00,0.000000e+00,0.000188,0.000014,...,0.0,0.0,0.0,0.002377,0.0,0.0,0.0,0.0,Calu3,Mock
29,0.000000e+00,0.000007,0.000021,1.584979e-06,1.132129e-07,1.358554e-06,1.132129e-07,9.057027e-07,0.000249,0.000018,...,0.0,0.0,0.0,0.001911,0.0,0.0,0.0,0.0,Calu3,Mock
30,0.000000e+00,0.000004,0.000022,1.498192e-06,7.490960e-07,1.098674e-06,0.000000e+00,4.993974e-07,0.000220,0.000012,...,0.0,0.0,0.0,0.002066,0.0,0.0,0.0,0.0,Calu3,Mock
31,9.577992e-08,0.000005,0.000026,1.340918e-06,1.053579e-06,1.149359e-06,0.000000e+00,2.873397e-07,0.000125,0.000010,...,0.0,0.0,0.0,0.001403,0.0,0.0,0.0,0.0,Calu3,SARS-CoV-2
32,0.000000e+00,0.000006,0.000024,2.121203e-06,7.070680e-07,1.767670e-07,0.000000e+00,8.838350e-07,0.000109,0.000011,...,0.0,0.0,0.0,0.001654,0.0,0.0,0.0,0.0,Calu3,SARS-CoV-2
33,0.000000e+00,0.000004,0.000025,2.135031e-06,8.354474e-07,9.282748e-07,0.000000e+00,5.569650e-07,0.000123,0.000010,...,0.0,0.0,0.0,0.001335,0.0,0.0,0.0,0.0,Calu3,SARS-CoV-2


#### 2. Regression models

In [26]:
#creating the result dataframes
results_A549 = pd.DataFrame(index=data_A549.columns[0:-2], #new data frame's indeces = gene names
                       columns=['Pval', 'A549_param'])

results_Calu = pd.DataFrame(index=data_Calu.columns[0:-2], #new data frame's indeces = gene names
                       columns=['Pval', 'Calu_param'])

In [27]:
## OLS for each genes for loop by the two cell-types
### Sometimes I goot a warning, could not figure out the reason
 
# A549
for gene in data_A549.columns[0:-2]: #we only need the genes, not the cell and treatment columns
  model = smf.ols(gene + ' ~ Treatment', data=data_A549).fit() # we fit the model to each gene
  results_A549.loc[gene] = model.pvalues['Treatment[T.SARS-CoV-2]'], model.params['Treatment[T.SARS-CoV-2]'] #results for each gene p values and params
    
# Calu3    
for gene in data_Calu.columns[0:-2]: 
  model = smf.ols(gene + ' ~ Treatment', data=data_Calu).fit() 
  results_Calu.loc[gene] = model.pvalues['Treatment[T.SARS-CoV-2]'], model.params['Treatment[T.SARS-CoV-2]']   

C:\Users\User\anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\User\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning:

invalid value encountered in less_equal



In [28]:
results_A549 #18544 rows
results_A549 = results_A549.dropna() #checking for missing values and removing them
results_A549 #17596 rows remained

,Pval,A549_param
DDX11L1,0.146303,2.23549e-08
WASH7P,0.185371,-3.35509e-06
LOC729737,0.238032,-1.4662e-06
LOC100133331,0.144478,-1.41115e-06
LOC100288069,0.691823,1.36896e-07
...,...,...
CD24,0.0853406,-0.000148158
BCORP1,0.502018,8.8808e-09
KDM5D,0.270786,-8.67676e-06
EIF1AY,0.464055,-2.55577e-06


In [29]:
results_Calu #15 544 rows
results_Calu = results_Calu.dropna() #removing missing values
results_Calu #16 451 rows remained

,Pval,Calu_param
DDX11L1,0.373901,3.19266e-08
WASH7P,0.485579,-7.15447e-07
LOC729737,0.0812646,6.29944e-06
LOC100133331,0.188752,5.6478e-07
LOC100288069,0.0853329,5.77928e-07
...,...,...
IL9R,0.373901,3.19266e-08
RPS4Y1,0.373901,6.38533e-08
ZFY,0.373901,3.19266e-08
PRKY,0.373901,6.38533e-08


In [30]:
#creating a column from the row names (gen names)
results_Calu.index.name = 'gen'
results_Calu.reset_index(inplace=True)
results_Calu.head()

,gen,Pval,Calu_param
0,DDX11L1,0.373901,3.19266e-08
1,WASH7P,0.485579,-7.15447e-07
2,LOC729737,0.0812646,6.29944e-06
3,LOC100133331,0.188752,5.6478e-07
4,LOC100288069,0.0853329,5.77928e-07


In [31]:
results_A549.index.name = 'gen1'
results_A549.reset_index(inplace=True)
results_A549.head()

,gen1,Pval,A549_param
0,DDX11L1,0.146303,2.23549e-08
1,WASH7P,0.185371,-3.35509e-06
2,LOC729737,0.238032,-1.4662e-06
3,LOC100133331,0.144478,-1.41115e-06
4,LOC100288069,0.691823,1.36896e-07


In [32]:
#merge the two cell-types
#keeping genes with observations for both cell-types
merged_results = results_Calu.merge(results_A549, left_on="gen", right_on='gen1')
merged_results #15934 colums remained

,gen,Pval_x,Calu_param,gen1,Pval_y,A549_param
0,DDX11L1,0.373901,3.19266e-08,DDX11L1,0.146303,2.23549e-08
1,WASH7P,0.485579,-7.15447e-07,WASH7P,0.185371,-3.35509e-06
2,LOC729737,0.0812646,6.29944e-06,LOC729737,0.238032,-1.4662e-06
3,LOC100133331,0.188752,5.6478e-07,LOC100133331,0.144478,-1.41115e-06
4,LOC100288069,0.0853329,5.77928e-07,LOC100288069,0.691823,1.36896e-07
...,...,...,...,...,...,...
15930,IL9R,0.373901,3.19266e-08,IL9R,0.224113,-9.56916e-08
15931,RPS4Y1,0.373901,6.38533e-08,RPS4Y1,0.343336,-3.85756e-05
15932,ZFY,0.373901,3.19266e-08,ZFY,0.52384,-5.78649e-07
15933,PRKY,0.373901,6.38533e-08,PRKY,0.340474,-2.51384e-06


#### 3. Results

In [33]:
#scatterplott of the results (coefficients)
fig = px.scatter(merged_results, x='A549_param', y='Calu_param', hover_data = ['gen'])
fig.show()

In [34]:
#comparing the coefficients of the two cell-types by a t-test
ttest_ind(merged_results['A549_param'], merged_results['Calu_param'])

Ttest_indResult(statistic=0.10697518076397394, pvalue=0.9148093430141571)

__Analysis__
* Results showed that the reaction to covid was not significantly different between cells A549 and Calu3. The p-value of the t-test was not significant.
* There are some genes that had different reactions across the two cell types ( in the upper left and lower right quadrant of the scatterplot). Some examples include: EEF1A1, TGFBI,LDHA, PGK1, VIM, CANX, NEAT1, PLEC, ELF3